<a href="https://colab.research.google.com/github/SomaKorada07/ERA3/blob/main/ERA3_Session_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
#         self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
#         self.pool1 = nn.MaxPool2d(2, 2)
#         self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
#         self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
#         self.pool2 = nn.MaxPool2d(2, 2)
#         self.conv5 = nn.Conv2d(256, 512, 3)
#         self.conv6 = nn.Conv2d(512, 1024, 3)
#         self.conv7 = nn.Conv2d(1024, 10, 3)

#     def forward(self, x):
#         x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
#         x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
#         x = F.relu(self.conv6(F.relu(self.conv5(x))))
#         x = F.relu(self.conv7(x))
#         x = x.view(-1, 10)
#         return F.log_softmax(x)

In [24]:
dropout_value = 0.01
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
              nn.Conv2d(1, 8, 3, padding=0, bias=False),
              nn.BatchNorm2d(8),
              nn.ReLU(inplace=True),
              nn.Dropout(dropout_value)) # output_size = 26, RF = 3
        self.conv2 = nn.Sequential(
              nn.Conv2d(8, 12, 3, padding=0, bias=False),
              nn.BatchNorm2d(12),
              nn.ReLU(inplace=True),
              nn.Dropout(dropout_value)) # output_size = 24, RF = 5
        self.conv3 = nn.Sequential(
              nn.MaxPool2d(kernel_size=2, stride=2), # output_size = 12, RF = 6
              nn.Conv2d(12, 8, 1, padding=0, bias=False),
              nn.BatchNorm2d(8),
              nn.ReLU(inplace=True),
              nn.Dropout(dropout_value)) # output_size = 12, RF = 6
        self.conv4 = nn.Sequential(
              nn.Conv2d(8, 12, 3, padding=0, bias=False),
              nn.BatchNorm2d(12),
              nn.ReLU(inplace=True),
              nn.Dropout(dropout_value)) # output_size = 10, RF = 10
        self.conv5 = nn.Sequential(
              nn.Conv2d(12, 16, 3, padding=0, bias=False),
              nn.BatchNorm2d(16),
              nn.ReLU(inplace=True),
              nn.Dropout(dropout_value)) # output_size = 8, RF = 14
        self.conv6 = nn.Sequential(
              nn.Conv2d(16, 32, 3, padding=0, bias=False),
              nn.BatchNorm2d(32),
              nn.ReLU(inplace=True),
              nn.Dropout(dropout_value)) # output_size = 6, RF = 18
        self.gap = nn.AvgPool2d(kernel_size=6) # output_size = 1, RF = 28
        self.conv7 = nn.Sequential(
              nn.Conv2d(32, 10, 1, padding=0, bias=False)) # output_size = 1, RF = 28

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.gap(x)
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [25]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
       BatchNorm2d-2            [-1, 8, 26, 26]              16
              ReLU-3            [-1, 8, 26, 26]               0
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 12, 24, 24]             864
       BatchNorm2d-6           [-1, 12, 24, 24]              24
              ReLU-7           [-1, 12, 24, 24]               0
           Dropout-8           [-1, 12, 24, 24]               0
         MaxPool2d-9           [-1, 12, 12, 12]               0
           Conv2d-10            [-1, 8, 12, 12]              96
      BatchNorm2d-11            [-1, 8, 12, 12]              16
             ReLU-12            [-1, 8, 12, 12]               0
          Dropout-13            [-1, 8, 12, 12]               0
           Conv2d-14           [-1, 12,

<ipython-input-24-80e98a4432cd>:50: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [26]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [27]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    train_losses = []
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        train_losses.append(loss)
        loss.backward()
        optimizer.step()
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Epoch={epoch} Accuracy={100*correct/processed:0.2f}')
        # pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [30]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.075, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-24-80e98a4432cd>:50: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
Loss=0.1561211198568344 Batch_id=468 Epoch=1 Accuracy=91.99: 100%|██████████| 469/469 [00:17<00:00, 26.40it/s]



Test set: Average loss: 0.0558, Accuracy: 9846/10000 (98.4600%)



Loss=0.03485864773392677 Batch_id=468 Epoch=2 Accuracy=98.22: 100%|██████████| 469/469 [00:18<00:00, 25.16it/s]



Test set: Average loss: 0.0397, Accuracy: 9870/10000 (98.7000%)



Loss=0.13222916424274445 Batch_id=468 Epoch=3 Accuracy=98.63: 100%|██████████| 469/469 [00:17<00:00, 27.39it/s]


...


Test set: Average loss: 0.0243, Accuracy: 9922/10000 (99.2200%)



Loss=0.0024756412021815777 Batch_id=468 Epoch=19 Accuracy=99.43: 100%|██████████| 469/469 [00:18<00:00, 25.07it/s]



Test set: Average loss: 0.0180, Accuracy: 9933/10000 (99.3300%)



Loss=0.00908786989748478 Batch_id=468 Epoch=20 Accuracy=99.52: 100%|██████████| 469/469 [00:18<00:00, 24.89it/s]



Test set: Average loss: 0.0189, Accuracy: 9941/10000 (99.4100%)

